In [841]:
import pandas as pd
from os import remove
from datetime import datetime

from classes import dbAzure
DBAzure = dbAzure.DBAzure

from helpers import correoEnvioArchivo, validarProduccion, rutas, correosDestino
estadoProduccion    = validarProduccion.validarProduccion()
rutaRelativa        = rutas.rutas()

from helpers import rutas, correoEnvioArchivo, correosDestino, validarProduccion
correoEnvioArchivo  = correoEnvioArchivo.correoEnvioArchivo

In [842]:
rutaRelativa = rutaRelativa.replace('\\SRC', '')
rutaRelativa

'/Users/camilo/Desktop/PROYECTOS'

In [843]:
dbAzure = DBAzure(estadoProduccion)

CONEXION CON EL SERVIDOR


### Obtengo la información de los equipos por enviar

In [844]:
equipos = dbAzure.getQuerySolpeds()
#solpeds
display(equipos)

/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


,id,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones,created_at,updated_at
0,338,8,-18,-12,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,NO OPERATIVO,0,prueba,,,,POR_ENVIAR,3778731,,,2023-03-22 11:23:40.000,2023-03-22 11:23:40.000
1,90,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,NO OPERATIVO,494,,None,02/02/2023 6:14,None,POR_ENVIAR,300100,20/02/2023 12:02,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
2,85,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,NO OPERATIVO,489,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,1002,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
3,82,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,NO OPERATIVO,486,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,150250,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250


In [845]:
equipos_envio = equipos.drop(['temperaturaMinima', 'OTRO', 'ESTADO', 'Observaciones','ESTADO_CORREO','temperaturaMaxima','Creado', 'DESCRIPCION', 'Modificado','created_at', 'updated_at', 'id' ], axis=1)
#display(equipos_envio)

### Cambiar nombre de columnas

In [846]:
equipos_envio = equipos_envio.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'nombreLocal'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO'
    })
#display(equipos_envio)


### Cambiar nombre de local

In [847]:
#equipos_envio['NOMBRE LOCAL'] = equipos_envio['NOMBRE LOCAL'].apply(lambda x: x.split('-')[1])
#display(equipos_envio)

### Cambiar datos de Local, tipo de equipo, de columnas

In [848]:
unique_values = equipos_envio['NOMBRE LOCAL'].unique()
unique_values


array(['CHJ5118800 - DARKSTORE COSTANERA',
       'CHJ5028800 - DARKSTORE KENNEDY'], dtype=object)

In [849]:
for i in unique_values:
    print(i)

CHJ5118800 - DARKSTORE COSTANERA
CHJ5028800 - DARKSTORE KENNEDY


# Obtención de data historica
# *Obtención de data historica*
# **Obtención de data historica**

In [850]:
equiposHistorico = dbAzure.getQuerySolpedsHistorico()
#display(equiposHistorico)
print(equiposHistorico.columns)


/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


Index(['id', 'Local', 'nombreMaquina', 'NUMERO_SERIE', 'Temperatura',
       'Creado_por', 'HORA', 'FechaReal', 'FECHA', 'nombreProducto',
       'temperaturaProducto', 'AC1', 'encargadoAC', 'TIPO_EQUIPO', 'Creado',
       'ZONA', 'EQUIPO', 'ESTADO', 'ANO', 'MES', 'DIA', 'CAMBIO_ESTADO',
       'nombreProducto2', 'temperaturaProducto2', 'CANTIDAD_FALLOS',
       'created_at', 'updated_at'],
      dtype='object')


In [851]:
equiposHistorico = equiposHistorico.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'Local'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO',
        'Observaciones' : 'TIPO EQUIPO',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
    })
#display(equiposHistorico)

In [852]:
equiposHistorico = equiposHistorico.drop(['CANTIDAD_FALLOS', 'NUMERO SERIE'], axis=1)


In [853]:
print(equiposHistorico['ESTADO'].unique())

['*' 'PENDIENTE DE REVISIÓN' 'NO OPERATIVO' 'None']


In [854]:
equipos_envio.dtypes

NUMERO MAQUINA     int64
NOMBRE LOCAL      object
TIPO EQUIPO       object
ZONA              object
NUMERO SERIE      object
OBSERVACION       object
MODELO            object
ORDEN TRABAJO     object
dtype: object

In [855]:
equiposHistorico.dtypes

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                  object
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object

# Quitar None

In [856]:
equiposHistorico = equiposHistorico.dropna(subset=['NUMERO MAQUINA'])

In [857]:
df_filtrado = equiposHistorico.loc[(equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

# Tranforma a Int64 la columna 'NUMERO MAQUINA'

In [858]:
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].replace(regex=r'[^0-9\-]+', value=-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].fillna(-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].astype('int64')

tipos_de_datos = equiposHistorico.dtypes
print(tipos_de_datos)

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                   int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object


# No operativos en los 2 locales

In [859]:
df_filtrado = equiposHistorico.loc[((equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (equiposHistorico['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)


# Unir las tablas 

In [860]:
base_equipos = equipos_envio.join(equiposHistorico.set_index(['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA']), on=['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA'], how='left')
#display(base_equipos)

In [861]:
df_filtrado = base_equipos.loc[((base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (base_equipos['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [862]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        61
None                     14
NO OPERATIVO              5
PENDIENTE DE REVISIÓN     2
dtype: int64


In [863]:
df_filtrado = base_equipos.loc[(base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [864]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

In [865]:
tipos_de_datos = base_equipos.dtypes
print(tipos_de_datos)

NUMERO MAQUINA                   int64
NOMBRE LOCAL                    object
TIPO EQUIPO                     object
ZONA                            object
NUMERO SERIE                    object
OBSERVACION                     object
MODELO                          object
ORDEN TRABAJO                   object
id                               int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO            

# Eliminar columnas

In [866]:
base_equipos = base_equipos.drop([
        'id',
        'Temperatura',
        'Creado_por',
        'Creado',
        'nombreProducto','temperaturaProducto', 'AC1',
        'Creado',
        'temperaturaProducto2',
        'created_at',
        'encargadoAC',
        'updated_at',
        'CAMBIO_ESTADO',
        'TIPO_EQUIPO',
        'nombreProducto2',
        'FechaReal',
        'HORA',
        'EQUIPO',
        'FECHA',
        'MODELO'
    ], axis=1)
#display(base_equipos)

In [867]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

# Se ordenan los datos por local, zona y número de máquina

In [868]:
base_equipos.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
#pd.set_option('display.max_rows', None)
#display(base_equipos)

In [869]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        61
None                     14
NO OPERATIVO              5
PENDIENTE DE REVISIÓN     2
dtype: int64


In [870]:
df_filtrado = base_equipos.loc[base_equipos['ESTADO'] == 'NO OPERATIVO']
#display(df_filtrado)

In [871]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,ANO,MES,DIA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023,5,20
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2020,1,27
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023,3,20
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2020,1,27
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023,4,4


# Crear columna fecha

In [872]:
base_equipos['FECHA'] = pd.to_datetime(base_equipos[['ANO', 'MES', 'DIA']].apply(lambda x: '-'.join(x.astype(str)), axis=1))
base_equipos = base_equipos.drop(['ANO', 'MES','DIA'], axis=1)
#display(base_equipos)


# Se añade un peso a cada registro (No operativo: 1, Otro:0)

In [873]:
base_equipos['CUENTA_ESTADO'] = base_equipos['ESTADO'].apply(lambda x: 1 if x == 'NO OPERATIVO' else 0)
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0


# Ordenizar por fechas y promediar ponderador de No Operativo

In [874]:
#cantidades = base_equipos.groupby('FECHA')['CUENTA_ESTADO'].transform('mean')
base_equipos['PONDERADOR NO OP'] = base_equipos.groupby(['FECHA', 'NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'])['CUENTA_ESTADO'].transform('mean')
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000


In [875]:
base_equipos = base_equipos.sort_values(by='FECHA')
display(base_equipos)


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000


# NO OPERATIVOS

In [876]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2020-01-27,1,0.100000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2020-01-27,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-20,1,1.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0.333333
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-05-20,1,1.000000


# Eliminar registros sobrantes del día (solo se deja un registro por día)

In [877]:
base_equipos.drop_duplicates(subset=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA', 'NUMERO SERIE','FECHA'], inplace=True)
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2020-01-27,1,1.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-03-01,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-07,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-10,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-20,1,1.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,*,2023-03-22,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,*,2023-03-22,0,0.000000


In [879]:
display(base_equipos[base_equipos['ZONA'] == 'Frutas y Verduras refrigeradas'])


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2020-01-27,1,1.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-20,1,1.0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,*,2023-03-22,0,0.0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,*,2023-03-22,0,0.0


### Tipos de 'ESTADO'

In [799]:
print(base_equipos['ESTADO'].unique())

['None' 'NO OPERATIVO' 'PENDIENTE DE REVISIÓN' '*']


In [801]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2020-01-27,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-20,1,1.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0.333333
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-05-20,1,1.000000


# Equipos de envío base

In [800]:
equipos_envio.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731



# Se añade las columnas para el reporte

In [685]:
equipos_envio['DIAS MALOS HISTORICO'] = 0
equipos_envio['CANTIDAD FALLAS'] = 0
equipos_envio['DIAS ULTIMA FALLA'] = 0
#display(equipos_envio)

# Se obtiene la cantidad de registros no operativos

In [687]:
cantidades = base_equipos.loc[base_equipos['PONDERADOR NO OP'] > 0].groupby(['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA' ])['ESTADO'].count()
print(cantidades)

NUMERO MAQUINA  NOMBRE LOCAL                      ZONA                          
2               CHJ5028800 - DARKSTORE KENNEDY    Fiambres y Quesos                 2
                                                  Frutas y Verduras refrigeradas    1
5               CHJ5028800 - DARKSTORE KENNEDY    Frutas y Verduras refrigeradas    1
8               CHJ5118800 - DARKSTORE COSTANERA  Congelados                        1
Name: ESTADO, dtype: int64


# Añadir cantidad de días no operativos a base

In [118]:
equipos_envio['DIAS MALOS HISTORICO'] = cantidades.values
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,0,0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,0,0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,1,0,0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,0,0


# Contabilizar los PENDIENTE DE APROBACIÓN a NO OPERATIVO

In [ ]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

In [ ]:
display(base_equipos)

In [ ]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO' or base_equipos['ESTADO'] == 'PENDIENTE DE REVISIÓN'])

In [ ]:
base_equipos.value_counts('ESTADO')

In [ ]:
df_filtrado = base_equipos.loc[(base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & ((base_equipos['ESTADO'] == 'NO OPERATIVO') | (base_equipos['ESTADO'] == 'PENDIENTE DE REVISIÓN'))]
display(df_filtrado)

# **---------------------------------------------------------------------------**

In [ ]:
rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION/DOCS')

In [ ]:
equipos_envio.to_excel("EQUIPOS_NO_OPERATIVOS.xlsx")
#remove("EQUIPOS_NO_OPERATIVOS.xlsx")


In [ ]:
rutaRelativa = rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION')
rutaRelativa


In [ ]:
correos         = correosDestino.correosDestino(estadoProduccion)
correos['to'].append('camilomansillaulloa@gmail.com')
correos

In [ ]:
archivos    = []
archivos.append({
    'nombre_archivo':   'EQUIPOS_NO_OPERATIVOS.xlsx',
    'tipo_archivo':     'application',
    'formato_archivo': 'vnd.ms-excel',
    'ruta':             f'{rutaRelativa}/EQUIPOS_NO_OPERATIVOS.xlsx'
})
correoEnvioArchivo('EQUIPOS NO OPERATIVOS DS', "", correos, archivos)

In [ ]:
equipos['ESTADO_CORREO'] = 'NO ENVIAR'
display(equipos)

In [ ]:
for index in equipos.index:
    dbAzure.changeStatusSolped(equipos.iloc[index])

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/CorreoGerenteDS.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()
#htmlBase

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/correoAprobacionGerenteBanderas.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()

In [ ]:
solpeds = dbAzure.getQuerySolpeds()
solpeds